In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from glob import glob
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

torch.backends.cudnn.benchmark = True


In [ ]:
import os

BASE_DIR = os.getcwd() # ноутбук должен находиться в папке с данными 

train_pq_path = os.path.join(BASE_DIR, "train_data")
test_pq_path = os.path.join(BASE_DIR, "test_data")

train_target_path = os.path.join(BASE_DIR, "train_target.csv")
test_target_path = os.path.join(BASE_DIR, "test_target.csv")

out_dir_train = os.path.join(BASE_DIR, "processed_data_train")
os.makedirs(out_dir_train, exist_ok=True)
out_dir_test = os.path.join(BASE_DIR, "processed_data_test")
os.makedirs(out_dir_test, exist_ok=True)
print("BASE_DIR:", BASE_DIR)

BASE_DIR: a:\alfa-masters


In [ ]:
def generate_npz_from_parquet(pq_path: str,
                              out_dir: str,
                              target_path: str = None):
    """
    Поочерёдно обрабатывает все parquet-файлы в папке pq_path и сохраняет для каждого
    блока npz-файл с ключами 'ids', 'sequences', 'targets' (если target_path задан).
    ---
    pq_path:    путь к папке с .parquet-файлами (train или test).
    out_dir:    куда сохранять npz (например, TRAIN_NPZ_DIR или TEST_NPZ_DIR).
    target_path: если указан, грузим CSV с таргетами и добавляем в npz.
    ---
    Возвращает:
      cardinalities: список кардинальностей для каждой enc_col_*
      max_rn:        максимальный rn среди всех паркет-файлов
    """
    print(f"\n[1] Генерация NPZ из: {pq_path}")
    files = sorted(glob(os.path.join(pq_path, "*.pq")))
    if len(files) == 0:
        raise RuntimeError(f"Не найдено parquet-файлов в {pq_path}")

    # Если задан target_path — прочитаем метки
    targets_map = {}
    if target_path is not None:
        y_df = pd.read_csv(target_path)
        targets_map = dict(zip(y_df["id"], y_df["target"]))
        print(f"[1.1] Загрузка таргетов из {target_path}: {len(targets_map)} записей")

    feat_cols = None
    cardinalities = None
    max_rn_total = 0

    for idx, fp in enumerate(files):
        print(f"\n[1.{idx+2}] Обрабатываю {fp}")
        df = pd.read_parquet(fp)
        print(f"     Shape = {df.shape}")

        # Определяем feat_cols и первичные cardinalities
        if feat_cols is None:
            feat_cols = [c for c in df.columns if c.startswith("enc_col_")]
            cardinalities = [df[c].nunique() + 1 for c in feat_cols]
            print(f"     Количество категориальных признаков: {len(feat_cols)}")
            print(f"     Первичные кардинальности: {cardinalities[:5]} ...")
        else:
            # Обновляем кардинальности, если в этом файле появились новые категории
            for i, c in enumerate(feat_cols):
                cardinalities[i] = max(cardinalities[i], df[c].nunique() + 1)

        # Сортируем и группируем по id
        df = df.sort_values(["id", "rn"])
        grouped = df.groupby("id", sort=False)

        seqs_block = []  # последовательности текущего файла
        ids_block = []   # айдишники
        tgts_block = []  # таргеты
        max_rn_block = 0

        for client_id, grp in grouped:
            rows = grp[["rn"] + feat_cols].to_numpy(dtype=np.int64)
            # rows[i,0] = rn, rows[i,1:] = feat values
            rows = sorted(rows, key=lambda x: x[0])
            feats = np.vstack([r[1:] for r in rows])  # (T_i, num_feats)
            seqs_block.append(feats)
            ids_block.append(client_id)
            if target_path is not None:
                tgts_block.append(targets_map.get(client_id, 0))
            max_rn_block = max(max_rn_block, rows[-1][0])

        # Сохраняем один .npz-файл
        save_path = os.path.join(out_dir, f"sequences_part_{idx}.npz")
        np.savez_compressed(
            save_path,
            ids=np.array(ids_block, dtype=np.int64),
            sequences=np.array(seqs_block, dtype=object),
            targets=np.array(tgts_block, dtype=np.int64) if target_path is not None else None
        )
        print(f"     Сохранено: {save_path}  ({len(seqs_block)} клиентов)")
        max_rn_total = max(max_rn_total, max_rn_block)

    print(f"\n[✔] Генерация завершена. Всего файлов: {len(files)}")
    print(f"    Максимальный rn среди всех файлов: {max_rn_total}")
    return cardinalities, max_rn_total




In [6]:
cardinalities, max_rn_total = generate_npz_from_parquet(train_pq_path, out_dir_train, train_target_path)
_, _ = generate_npz_from_parquet(test_pq_path, out_dir_test)


[1] Генерация NPZ из: a:\alfa-masters\train_data
[1.1] Загрузка таргетов из a:\alfa-masters\train_target.csv: 3000000 записей

[1.2] Обрабатываю a:\alfa-masters\train_data\train_data_0.pq
     Shape = (1974724, 59)
     Количество категориальных признаков: 57
     Первичные кардинальности (пример): [3, 7, 3, 9, 8] ...
     Сохранено: a:\alfa-masters\processed_data_train\sequences_part_0.npz  (250000 клиентов)

[1.3] Обрабатываю a:\alfa-masters\train_data\train_data_1.pq
     Shape = (2107305, 59)
     Сохранено: a:\alfa-masters\processed_data_train\sequences_part_1.npz  (250000 клиентов)

[1.4] Обрабатываю a:\alfa-masters\train_data\train_data_10.pq
     Shape = (2296372, 59)
     Сохранено: a:\alfa-masters\processed_data_train\sequences_part_2.npz  (250000 клиентов)

[1.5] Обрабатываю a:\alfa-masters\train_data\train_data_11.pq
     Shape = (2450630, 59)
     Сохранено: a:\alfa-masters\processed_data_train\sequences_part_3.npz  (250000 клиентов)

[1.6] Обрабатываю a:\alfa-masters\tra

In [13]:
def load_sequences_from_npz(npz_dir: str):
    """
    Загружает из папки npz_dir все файлы sequences_part_*.npz и 
    возвращает три списка:
      sequences:  список numpy-массивов (каждый shape=(T_i, F))
      targets:    список меток (если в npz есть 'targets'), иначе None
      ids:        список всех id подряд
    Для train npz в каждом есть ключ 'targets', для test — targets=None.
    """
    files = sorted(glob(os.path.join(npz_dir, "sequences_part_*.npz")))
    if len(files) == 0:
        raise RuntimeError(f"Не найдено NPZ-файлов в {npz_dir}")

    sequences = []
    targets = []
    for fp in files:
        data = np.load(fp, allow_pickle=True)
        data = np.load(fp, allow_pickle=True)
        sequences.extend(data["sequences"])
        targets.extend(data["targets"])

    return sequences, targets


In [8]:
class PanelDataset(Dataset):
    def __init__(self, sequences, targets=None, max_len=None):
        self.sequences = sequences
        self.targets = targets
        self.max_len = max(len(s) for s in sequences) if max_len is None else max_len
        print(f"[Dataset] max_len={self.max_len}, samples={len(sequences)}")
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        # Return raw data without padding to move padding to collate_fn
        item = {'seq': self.sequences[idx]}
        if self.targets is not None:
            item['y'] = self.targets[idx]
        return item

def collate_fn(batch, max_len):
    sequences = [item['seq'] for item in batch]
    padded_seqs = []
    masks = []

    for seq in sequences:
        # ПАДДИНГ В КОНЦЕ ПОСЛЕДОВАТЕЛЬНОСТИ
        pad_len = max_len - len(seq)
        padded = np.zeros((max_len, seq.shape[1]), dtype=np.int64)
        padded[:len(seq)] = seq  # реальные данные в начале
        
        # Маска: 1 для реальных данных, 0 для паддинга
        mask = np.zeros(max_len, dtype=np.bool_)
        mask[:len(seq)] = True
        
        padded_seqs.append(padded)
        masks.append(mask)
    
    batch_x = torch.LongTensor(np.stack(padded_seqs))
    batch_mask = torch.BoolTensor(np.stack(masks))
    
    if 'y' in batch[0]:
        batch_y = torch.LongTensor([item['y'] for item in batch])
        return {'x': batch_x, 'mask': batch_mask, 'y': batch_y}
    
    return {'x': batch_x, 'mask': batch_mask}

In [15]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, cardinalities, max_len,
                 d_model=256, nhead=8, num_layers=4, dropout=0.1):
        super().__init__()
        # REVERT to standard Embedding layers
        self.embs = nn.ModuleList([
            nn.Embedding(card, d_model, padding_idx=0)
            for card in cardinalities
        ])
        self.pos_emb = nn.Embedding(max_len + 1, d_model) # +1 для [CLS]-токена
        
        self.d_model = d_model

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=4*d_model,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model, 2)
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        self.input_ln = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        
        B, L, _ = x.shape  # Batch size, Sequence length
        d_model = self.d_model
        x_emb = torch.zeros(B, L, d_model, device=x.device)
        
        for i, emb in enumerate(self.embs):
            
            feature = x[..., i]  # Shape [B, L]
            x_emb += emb(feature)  # Accumulate embeddings
        
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x_emb = torch.cat((cls_tokens, x_emb), dim=1)  # [B, L+1, D]
        
        # Позиционные эмбеддинги (включая [CLS])
        pos = torch.arange(0, L + 1, device=x.device)  
        x_emb = x_emb + self.pos_emb(pos).unsqueeze(0)
        
        
        x_emb = self.input_ln(x_emb)
        
        
        cls_mask = torch.ones(B, 1, dtype=torch.bool, device=mask.device)
        mask = torch.cat((cls_mask, mask), dim=1)  # [B, L+1]
        
        
        out = self.transformer(x_emb, src_key_padding_mask=~mask)
        
        
        cls_output = out[:, 0]
        return self.fc(self.norm(cls_output))



In [ ]:
def train_loop(model, dl_train, dl_val, device, epochs=10, lr=1e-3):
    best_auc = 0
    best_state = None
    patience = 6
    epochs_no_improve = 0
    save_path = 'best_model.pth'
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',       
        factor=0.5,       
        patience=2,       
        threshold=0.001,  
        min_lr=1e-6       
    )

    class_weights = torch.tensor([1.0, 5.0], dtype=torch.float32, device=device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    scaler = torch.amp.GradScaler('cuda')  # Mixed precision
    
    for epoch in range(1, epochs+1):
        start_time = time.time()
        model.train()
        total_loss = 0
        
        for batch_idx, batch in enumerate(dl_train, 1):
            x = batch['x'].to(device, non_blocking=True)
            mask = batch['mask'].to(device, non_blocking=True)
            y = batch['y'].to(device, non_blocking=True)
            
            optimizer.zero_grad(set_to_none=True)
            
            with torch.amp.autocast('cuda'):
                preds = model(x, mask)
                loss = criterion(preds, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            # УДАЛИТЬ: scheduler.step() здесь! 
            
            total_loss += loss.item()
            
            if batch_idx % 100 == 0:
                
                current_lr = optimizer.param_groups[0]['lr']
                print(f"  Batch {batch_idx}/{len(dl_train)} | Loss: {loss.item():.4f} | LR: {current_lr:.2e}")

        epoch_time = time.time() - start_time
        avg_loss = total_loss / len(dl_train)
        print(f"Epoch {epoch} | Train Loss: {avg_loss:.4f} | Time: {epoch_time:.1f}s")
        
        # Validation
        model.eval()
        all_preds, all_targets = [], []
        
        with torch.no_grad():
            for batch in dl_val:
                x = batch['x'].to(device, non_blocking=True)
                mask = batch['mask'].to(device, non_blocking=True)
                
                with torch.amp.autocast('cuda'):
                    preds = model(x, mask)
                
                all_preds.append(preds.cpu())
                all_targets.append(batch['y'])
        
        all_preds = torch.cat(all_preds)
        all_targets = torch.cat(all_targets)
        probs = torch.softmax(all_preds, dim=1)[:, 1]
        auc = roc_auc_score(all_targets.numpy(), probs.numpy())
        print(f"Validation AUC: {auc:.4f}\n")
        
        # Обновляем scheduler НА ОСНОВАНИИ AUC
        scheduler.step(auc) 
        
        if auc > best_auc:
            best_auc = auc
            best_state = model.state_dict()
            torch.save(best_state, save_path)
            epochs_no_improve = 0
            print(f"  >>> New best model saved with AUC: {best_auc:.4f}")
        else:
            epochs_no_improve += 1
            print(f"  No improvement for {epochs_no_improve} epochs.")

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered. Best AUC: {best_auc:.4f}")
            break

if __name__ == '__main__':
    seqs, targets = load_sequences_from_npz(out_dir_train)
    cards, max_rn = cardinalities, int(max_rn_total)
    seq_train, seq_val, y_train, y_val = train_test_split(seqs, targets, stratify=targets, test_size=0.2)
    
    # Create datasets and collate functions
    ds_train = PanelDataset(seq_train, y_train, max_len=max_rn)
    ds_val = PanelDataset(seq_val, y_val, max_len=max_rn)
    
    # Use custom collate with pinned memory
    collate_train = lambda b: collate_fn(b, ds_train.max_len)
    collate_val = lambda b: collate_fn(b, ds_val.max_len)
    
    #ctx = multiprocessing.get_context('spawn')
    
    dl_train = DataLoader(
        ds_train, 
        batch_size=512,
        shuffle=True,
        num_workers=0,  
        pin_memory=True,
        collate_fn=collate_train,
        
    )
    
    dl_val = DataLoader(
        ds_val,
        batch_size=1024,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_val,
        
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TimeSeriesTransformer(cards, max_len=max_rn, d_model=256, nhead=8, num_layers=2, dropout=0.3)

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    print(f"Using device: {device}")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
    
    train_loop(model, dl_train, dl_val, device, epochs=20, lr=5e-4)

In [23]:
seqs

[array([[ 1,  4,  1,  6,  1,  0,  1,  3,  1,  4,  3, 10,  0,  3,  1,  1,
          3, 16,  0,  3,  3,  3,  3,  3,  3,  8,  1,  2, 11,  0,  3,  1,
          3, 17,  3,  4,  4,  2,  1,  3,  2,  4,  3,  3,  3,  2, 16,  3,
          3,  3,  2,  3,  3,  0,  5,  3, 11],
        [ 1,  4,  1,  6,  1,  0,  1,  0,  1,  4,  0, 12,  0,  0,  1,  1,
          0, 16,  0,  0,  0,  0,  0,  0,  0,  8,  1,  2, 11,  0,  3,  1,
          3, 17,  0,  4,  1,  9,  1,  0,  2,  1,  0,  0,  0,  2, 16,  0,
          0,  0, 14, 14,  0,  0,  5,  0,  0],
        [ 1,  3,  1,  6,  1,  0,  1,  0,  1,  4,  0, 11,  1,  0,  1,  1,
          0, 16,  0,  0,  0,  0,  0,  0,  0,  8,  0,  2,  8,  1,  5,  1,
          2, 17,  0,  4,  1, 12,  1,  0,  0,  1,  0,  0,  0,  2, 15,  0,
          0,  0,  4,  8,  0,  0,  5,  0, 11],
        [ 1,  1,  1,  6,  1,  0,  1,  0,  1,  4,  3,  7,  0,  0,  1,  0,
          3, 16,  0,  0,  3,  3,  3,  3,  3,  8,  1,  2,  4,  0,  3,  1,
          3, 17,  3,  4,  1, 16,  1,  0,  6,  4,  0,  3,  3

In [ ]:
MODEL_PATH = 'best_model.pth'  # путь к сохранённой модели
cardinalities = [2, 8, 2, 17, 7, 2, 2, 4, 2, 5, 4, 16, 2, 4, 2, 2, 4, 20, 4, 4, 4, 4, 4, 4, 4, 20, 2, 4, 14, 2, 6, 4, 7, 20, 4, 5, 5, 17, 2, 4, 7, 5, 4, 4, 4, 20, 20, 4, 4, 4, 18, 17, 4, 4, 10, 4, 20]
PROCESSED_TEST_DIR = out_dir_test
SUBMISSION_PATH = "submission.csv"
BATCH_SIZE = 256
NUM_WORKERS = 0
MAX_LEN = 58

In [ ]:
class TestPanelDataset(Dataset):
    def __init__(self, npz_dir, cardinalities):
        files = sorted([os.path.join(npz_dir, fn) 
                       for fn in os.listdir(npz_dir) if fn.endswith(".npz")])
        self.cardinalities = cardinalities
        self.ids = []
        self.sequences = []
        
        for fp in files:
            data = np.load(fp, allow_pickle=True)
            self.ids.extend(data["ids"].tolist())
            self.sequences.extend(data["sequences"].tolist())
        
        print(f"[Test Dataset] Loaded {len(self.ids)} samples")

    def __len__(self):  # ДОБАВЛЕНО: метод для получения длины датасета
        return len(self.ids)

    def __getitem__(self, idx):
        seq = self.sequences[idx].copy()
        
        # Коррекция значений вне диапазона
        for i, card in enumerate(self.cardinalities):
            col = seq[:, i]
            # Заменяем значения >= card на 0
            col[col >= card] = 0
            seq[:, i] = col
        
        return {
            'id': self.ids[idx],
            'seq': seq  # Возвращаем непатченную последовательность
        }

def test_collate_fn(batch, max_len):
    """Collate function for inference batches"""
    ids = [item['id'] for item in batch]
    sequences = [item['seq'] for item in batch]
    
    padded_seqs = []
    masks = []
    
    for seq in sequences:
        pad_len = max_len - len(seq)
        # Паддинг в конце последовательности
        padded = np.zeros((max_len, seq.shape[1]), dtype=np.int64)
        padded[:len(seq)] = seq  # Реальные данные в начале
        
        # Создание маски
        mask = np.zeros(max_len, dtype=np.bool_)
        mask[:len(seq)] = True  # True только для реальных данных
        
        padded_seqs.append(padded)
        masks.append(mask)
    
    return {
        'id': ids,
        'x': torch.LongTensor(np.stack(padded_seqs)),
        'mask': torch.BoolTensor(np.stack(masks))
    }

def run_inference():


    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    # Создание датасета
    ds_test = TestPanelDataset(
        npz_dir=PROCESSED_TEST_DIR,
        cardinalities=cardinalities
    )
    
    # Создание collate_fn с фиксированным max_len
    from functools import partial
    collate_fn = partial(test_collate_fn, max_len=MAX_LEN)
    
    # DataLoader
    dl_test = DataLoader(
        ds_test,
        batch_size=512,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn,
        pin_memory=True
    )
    
    # Загрузка модели
    model = TimeSeriesTransformer(
        cardinalities=cardinalities,
        max_len=MAX_LEN,
        d_model=256,
        nhead=8,
        num_layers=2,
        dropout=0.0  # Отключаем dropout для инференса
    ).to(device)
    
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()
    
    # Инференс
    all_ids = []
    all_probs = []
    
    with torch.inference_mode():  
        for batch in dl_test:
            x = batch["x"].to(device, non_blocking=True)
            mask = batch["mask"].to(device, non_blocking=True)
            
            logits = model(x, mask)
            probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
            
            all_ids.extend(batch["id"])
            all_probs.extend(probs)
    
    
    
    df_sub = pd.DataFrame({"id": all_ids, "target": all_probs})
    df_sub.to_csv(SUBMISSION_PATH, index=False)
    print(f"Submission saved to {SUBMISSION_PATH} with {len(df_sub)} rows")

if __name__ == "__main__":
    
    run_inference()

Using device: cuda
[Test Dataset] Loaded 500000 samples
✅ Submission saved to submission_new_w.csv with 500000 rows
